<a href="https://colab.research.google.com/github/johnr0/InGen/blob/main/InGen_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing requirements

In [1]:
!pip install flask==2.0.1
!pip install flask_api==2.0
# !pip install python-socketio==4.6.0
# !pip install python-engineio==3.13.2
!pip install flask_socketio==5.1.0
!pip install flask_cors==3.0.3
# !pip install Werkzeug==0.16.1

!pip install diffusers==0.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install gevent-websocket


!pip install pyngrok
!ngrok authtoken 1pDdjhcWhyh8VUFRTYCnGgCOI8g_3f56gBgxvREzT4gfAWXp2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 133 kB 32.0 MB/s 
     |████████████████████████████████| 232 kB 72.6 MB/s 
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0.1
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
  Attempting uninstall: flask
    Found existing installation: Flask 1

# Logging in with Huggingface Token

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


# Installing Ngrok

In [3]:
# ngrok install
!wget --quiet https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


# Diffusion

In [1]:
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space. 
# vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae", use_auth_token=True)
vae = AutoencoderKL.from_pretrained("runwayml/stable-diffusion-v1-5", revision='fp16', torch_dtype=torch.float16, subfolder="vae", use_auth_token=True)

# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = text_encoder.half()

# 3. The UNet model for generating the latents.
# unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", use_auth_token=True)
unet = UNet2DConditionModel.from_pretrained("runwayml/stable-diffusion-v1-5", revision='fp16', torch_dtype=torch.float16, subfolder="unet", use_auth_token=True)

from diffusers import DDIMScheduler

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)

from torch import autocast
from tqdm.auto import tqdm
from PIL import Image

vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device) 

embedded_text_prompts = {}



def text_prompt_embed(t):
  with torch.no_grad():
    if t in embedded_text_prompts:
      return embedded_text_prompts[t]
    else: 
      text_input = tokenizer([t], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
      text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
      embedded_text_prompts[t] = text_embeddings
      return text_embeddings

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.0.self_attn.v_proj.weight', 'vision_model.encoder.layers.19.self_attn.q_proj.weight', 'vision_model.encoder.layers.9.mlp.fc2.bias', 'vision_model.encoder.layers.16.layer_norm1.bias', 'vision_model.encoder.layers.18.self_attn.k_proj.bias', 'vision_model.encoder.layers.18.self_attn.v_proj.weight', 'vision_model.encoder.layers.20.mlp.fc2.bias', 'vision_model.encoder.layers.6.self_attn.v_proj.weight', 'vision_model.encoder.layers.0.layer_norm1.weight', 'vision_model.encoder.layers.10.self_attn.out_proj.weight', 'vision_model.encoder.layers.21.self_attn.v_proj.bias', 'vision_model.encoder.layers.7.mlp.fc2.weight', 'vision_model.encoder.layers.20.self_attn.k_proj.bias', 'vision_model.encoder.layers.1.self_attn.out_proj.bias', 'vision_model.encoder.layers.15.self_attn.out_proj.bias', 'vision_model.encoder.layers.13.self_attn.q_proj.weight', 'vision

In [2]:
# variables that can change dynamically during generation
directional_prompt_inputs = {}

directional_prompt_inputs_proto = {}

prompt_inputs = {}

prompt_inputs_proto = {}

prompts_whole_proto = {}

guidance_scale = {}

threads = {}

sub_threads = {}

gen_stop = {}

latents_list = {}
dir_prompt_list = {}
prompt_list = {}
guidance_scale_list = {}
prompts_whole_list = {}

lms = []

# Initializing Ngrok

In [3]:


from pyngrok import ngrok

# ngrok.disconnect(ngrok_tunnel.public_url)
ngrok_tunnel = ngrok.connect(5001)
ngrok_tunnel

<NgrokTunnel: "http://5b1c-104-196-1-175.ngrok.io" -> "http://localhost:5001">

# Run server

In [4]:

from flask_socketio import SocketIO, emit
from flask import Flask, request, copy_current_request_context
from flask_cors import CORS
from random import gauss, random
# from threading import Thread, Event
from time import sleep
import time

import numpy as np
from PIL import Image
import base64
from io import BytesIO
import torch
from threading import Thread
from gevent.pywsgi import WSGIServer
from geventwebsocket.handler import WebSocketHandler

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'

socketio = SocketIO(app, cors_allowed_origins='*', async_mode='gevent', ping_interval=1, max_http_buffer_size=10e8)
CORS(app, resources={r"/*": {"origins": "*"}})

@app.route('/hello')
def hello():
    return "Hello World!"


# Handle the webapp connecting to the websocket
@socketio.on('connect')
def test_connect():
    print('someone connected to websocket:', request.sid)
    emit('connect', {'data': 'Connected! ayy'})
    
@socketio.on('disconnect')
def test_connect():
    print(request.sid)
    print('someone disconnected to websocket')
    emit('disconnect', {'data': 'Disconnected! ayy'})

# Handle the webapp connecting to the websocket, including namespace for testing
@socketio.on('connect', namespace='/devices')
def test_connect2():
    print('someone connected to websocket!')
    emit('responseMessage', {'data': 'Connected devices! ayy'})


def preprocess_mask(mask):
    mask = mask.convert("L")
    w, h = mask.size
    if w < h:
      h = int(h*(512/w))
      w = 512
    else:
      w = int(w*(512/h))
      h = 512
      

    # if w > 512:
    #   h = int(h * (512/w))
    #   w = 512
    # if h > 512:
    #   w = int(w*(512/h))
    #   h = 512
    
    w, h = map(lambda x: x - x % 64, (w, h)) 
    w //= 8
    h //= 8

    mask = mask.resize((w, h), resample=Image.LANCZOS)

    # mask = np.array(mask).astype(np.float32) / 255.0
    mask = np.array(mask).astype(np.float16) / 255.0
    mask = np.tile(mask, (4,1,1))
    mask = mask[None].transpose(0,1,2,3)
    mask[np.where(mask !=0.0)]=1.0
    mask = torch.from_numpy(mask)
    return mask

def preprocess(image):
    image = image.convert('RGB')
    w, h = image.size
    print(w,h)
    if w < h:
      h = int(h*(512/w))
      w = 512
    else:
      w = int(w*(512/h))
      h = 512
    # if w > 512:
    #   h = int(h * (512/w))
    #   w = 512
    # if h > 512:
    #   w = int(w*(512/h))
    #   h = 512
    w, h = map(lambda x: x - x % 64, (w, h))  # resize to integer multiple of 64, 32 can sometimes result in tensor mismatch errors

    image = image.resize((w, h), resample=Image.LANCZOS)
    print(image.size)
    # image = np.array(image).astype(np.float32) / 255.0
    image = np.array(image).astype(np.float16) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.0 * image - 1.0

def numpy_to_pil(images):
    """
    Convert a numpy image or a batch of images to a PIL image.
    """
    if images.ndim == 3:
            images = images[None, ...]
    images = (images * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image).convert('RGBA') for image in images]
    return pil_images

@socketio.on('test')
def handle_message_t(message):
    print(message)

@socketio.on('gen_stop')
def gen_stop_message_t(message):
    gen_stop[message['stroke_id']] = True

@socketio.on('guidance_scale_update')
def guidance_scale_message_t(message):
  guidance_scale[message['stroke_id']] = message['guidance_scale']

@socketio.on('prompts_update')
def prompts_message_t(message):
  text_prompts = message['text_prompts']
  text_prompt_weights = message['text_prompt_weights']

  text_prompt_embedding = None
  tpw = 0
  for tp_idx, text_prompt in enumerate(text_prompts):
    # print('inloop1', time.time()-now)
    cur_embedding = text_prompt_embed(text_prompt)
    # print('inloop2', time.time()-now)
    if text_prompt_embedding==None:
      text_prompt_embedding = cur_embedding*text_prompt_weights[tp_idx]
    else:
      text_prompt_embedding = text_prompt_embedding + cur_embedding*text_prompt_weights[tp_idx]
    tpw = tpw + text_prompt_weights[tp_idx]
    # print('inloop3', time.time()-now)
  text_prompt_embedding = text_prompt_embedding/tpw

  # uncond_embeddings = text_prompt_embed('')
  # text_embeddings = torch.cat([uncond_embeddings, text_prompt_embedding])

  prompts_whole_proto[message['stroke_id']] = message['prompts_proto']
  prompt_inputs[message['stroke_id']] = text_prompt_embedding #text_embeddings
  prompt_inputs_proto[message['stroke_id']] = message['selected_prompts_proto']

@socketio.on('directional_prompts_update')
def directional_prompts_message_t(message):
  directional_prompts = message['directional_prompts']

  directional_vector = None
  for directional_prompt in directional_prompts:
    if directional_prompt['value']==0:
      continue
    if directional_vector == None:
      directional_vector = float(directional_prompt['value'])/100.0 * (text_prompt_embed(directional_prompt['promptB'])-text_prompt_embed(directional_prompt['promptA']))
    else:
      directional_vector = directional_vector + float(directional_prompt['value'])/100.0 * (text_prompt_embed(directional_prompt['promptB'])-text_prompt_embed(directional_prompt['promptA']))
  directional_prompt_inputs[message['stroke_id']] = directional_vector
  directional_prompt_inputs_proto[message['stroke_id']] = message['directional_prompts_proto']        


@socketio.on('gen_start')
@torch.no_grad()
def handle_message(message):
    @copy_current_request_context
    def handle_message_thread(message):
      try:
        with torch.no_grad():
          now = time.time()

          gen_stop[message['stroke_id']] = False

          guidance_scale[message['stroke_id']] = message['guidance_scale']
          text_prompts = message['text_prompts']
          text_prompt_weights = message['text_prompt_weights']
          directional_prompts = message['directional_prompts']

          if message['gen_tick']==0:
            latents_list[message['stroke_id']] = []
            dir_prompt_list[message['stroke_id']] = []
            prompt_list[message['stroke_id']] = []
            guidance_scale_list[message['stroke_id']] = []
            prompts_whole_list[message['stroke_id']] = []
          else:
            # pop things until the tick
            latents_list[message['stroke_id']] = [None]*message['gen_tick']
            dir_prompt_list[message['stroke_id']] = [None]*message['gen_tick']
            prompt_list[message['stroke_id']] = [None]*message['gen_tick']
            guidance_scale_list[message['stroke_id']] = [None]*message['gen_tick']
            prompts_whole_list[message['stroke_id']] = [None]*message['gen_tick']

          layer_img_o = Image.open(BytesIO(base64.b64decode(message['layer_img'].split(",",1)[1])))
          area_img = Image.new("RGBA", layer_img_o.size, "WHITE")
          area_img_o = Image.open(BytesIO(base64.b64decode(message['area_img'].split(",",1)[1])))
          area_img.paste(area_img_o, (0,0), area_img_o)
          
          


          overcoat_ratio = message['overcoat_ratio']
          seed = message['seed']
          generator = torch.Generator(device='cuda', )
          generator.manual_seed(seed) 

          # set mask from area_img
          area_mask = preprocess_mask(area_img)
          area_mask = area_mask.to(torch_device)

          

          # add black or white background to the layer image
          layer_img_back = Image.new("RGBA", layer_img_o.size, "WHITE")
          layer_img_back.paste(layer_img_o, (0, 0), layer_img_o)
          layer_img_back.convert('RGB')
          layer_img = preprocess(layer_img_back)
          init_latents = vae.encode(layer_img.to(torch_device)).latent_dist.sample()
          init_latents = 0.18215 * init_latents

          init_latents = init_latents.half()

          noise = torch.randn(init_latents.shape, generator=generator, device=torch_device, dtype=torch.float16)

          num_inference_steps = message['steps']
          scheduler.set_timesteps(num_inference_steps)
          t_init = scheduler.timesteps[message['gen_tick']]
          # if message['gen_tick'] < int((1-overcoat_ratio)*num_inference_steps):
          layer_array = np.copy(np.asarray(layer_img_o))

          alphas = np.ones(layer_array[:,:,3,None].shape)*255
          layer_array = np.concatenate((layer_array[:,:,3,None], layer_array[:,:,3,None], layer_array[:,:,3,None], alphas), axis = 2)
          layer_array = np.array(layer_array, dtype = np.uint8)
      
          layer_img_mask = Image.fromarray(layer_array)
          layer_mask = preprocess_mask(layer_img_mask)
          layer_mask = layer_mask.to(torch_device)

          last_mask = None
          if 'last_img' in message:
            last_img = Image.new("RGBA", layer_img_o.size, "WHITE")
            last_img_o = Image.open(BytesIO(base64.b64decode(message['last_img'].split(",",1)[1])))
            last_img.paste(last_img_o, (0,0), last_img_o)
            last_mask = preprocess_mask(last_img)
            last_mask = last_mask.to(torch_device)
            last_mask = 1+last_mask-area_mask
            # last_mask = layer_mask

            last_latents = torch.Tensor(message['last_latents'])
            last_latents = last_latents.to(torch_device)
            # lms.append(last_latents)
            # lms.append(last_img)


          print(time.time()-now)
          if message['gen_tick']==0:   
            
            # latents = noise # start from the purse noise

            latents = (1-area_mask)* (1-layer_mask) * noise + (1-(1-area_mask)* (1-layer_mask)) * init_latents # In / Out
            latents = (0.1) * latents + 0.9 * noise
            
            # scheduler.add_noise(latents, noise, t-1)
          else:
            # print('stored latent is used')
            latents = torch.Tensor(message['latents'])
            latents = latents.to(torch_device)


          # print(latents.size())


          print(time.time()-now)

          # set directional prompt embeddings
          directional_vector = None
          for directional_prompt in directional_prompts:
            if directional_prompt['value']==0:
              continue
            if directional_vector == None:
              directional_vector = float(directional_prompt['value'])/100.0 * (text_prompt_embed(directional_prompt['promptB'])-text_prompt_embed(directional_prompt['promptA']))
            else:
              directional_vector = directional_vector + float(directional_prompt['value'])/100.0 * (text_prompt_embed(directional_prompt['promptB'])-text_prompt_embed(directional_prompt['promptA']))
          
          text_prompt_embedding = None
          tpw = 0
          for tp_idx, text_prompt in enumerate(text_prompts):
            # print('inloop1', time.time()-now)
            cur_embedding = text_prompt_embed(text_prompt)
            # print('inloop2', time.time()-now)
            if text_prompt_embedding==None:
              text_prompt_embedding = cur_embedding*text_prompt_weights[tp_idx]
            else:
              text_prompt_embedding = text_prompt_embedding + cur_embedding*text_prompt_weights[tp_idx]
            tpw = tpw + text_prompt_weights[tp_idx]
            # print('inloop3', time.time()-now)
          text_prompt_embedding = text_prompt_embedding/tpw

          uncond_embeddings = text_prompt_embed('')
          # if directional_vector!=None:
          #   text_embeddings = torch.cat([uncond_embeddings, text_prompt_embedding+directional_vector])
          # else:
          #   text_embeddings = torch.cat([uncond_embeddings, text_prompt_embedding])

          directional_prompt_inputs[message['stroke_id']] = directional_vector
          directional_prompt_inputs_proto[message['stroke_id']] = message['directional_prompts_proto']
          prompts_whole_proto[message['stroke_id']] = message['prompts_proto']
          prompt_inputs[message['stroke_id']] = text_prompt_embedding
          prompt_inputs_proto[message['stroke_id']] = message['selected_prompts_proto']
          # print(uncond_embeddings.size(), text_prompt_embedding.size())

          print(time.time()-now)

          # num inference steps should be fixed
          gen_duration = np.min([message['gen_tick']+message['gen_duration'], message['steps']])
          gen_duration = np.max([1, gen_duration])
          
          print(scheduler.timesteps, scheduler.timesteps[message['gen_tick']:])
          # sub_threads[message['stroke_id']] = Thread(target=sendoutIntermediate, args = (message['gen_tick'], layer_img_o, area_img_o, message['area_img'], message['stroke_id'], gen_duration))
          # sub_threads[message['stroke_id']].start()

          print(time.time()-now)

          
          for i, t in enumerate(scheduler.timesteps[message['gen_tick']:gen_duration]):
            print(i, 'start', guidance_scale[message['stroke_id']])
            if gen_stop[message['stroke_id']]:
              return
            # Do something about generation
            latent_model_input = torch.cat([latents] * 2)
            # predict the noise residual
            with torch.no_grad():
              if directional_prompt_inputs[message['stroke_id']] == None:
                text_embeddings = torch.cat([uncond_embeddings, prompt_inputs[message['stroke_id']]])
              else:
                text_embeddings = torch.cat([uncond_embeddings, prompt_inputs[message['stroke_id']]+directional_prompt_inputs[message['stroke_id']]])
              print('unet pipeline0', time.time()-now, latent_model_input.is_cuda, type(t), t.shape, text_embeddings.is_cuda)
              noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]
              print(noise_pred.type())
            # perform guidance
            print('unet pipeline1', time.time()-now)
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            
            noise_pred = noise_pred_uncond + guidance_scale[message['stroke_id']] * (noise_pred_text - noise_pred_uncond)
            latents = scheduler.step(noise_pred, t, latents)["prev_sample"]
            print('unet pipeline2', time.time()-now)

            # t_noise = torch.randn(latents.shape, device=torch_device)
            if t > 1:
              # when over overcoat ratio
              if last_mask == None:
                if i >= int((1-overcoat_ratio)*num_inference_steps):
                  # init_latents_proper = scheduler.add_noise(init_latents, noise, t-1)
                  proto_prod = torch.ones(1,1,1,1, device=torch_device, dtype=torch.float16)
                  sqrt_alpha_prod = scheduler.alphas_cumprod[t-1] ** 0.5
                  sqrt_one_minus_alpha_prod = (1 - scheduler.alphas_cumprod[t-1]) ** 0.5
                  sqrt_alpha_prod=sqrt_alpha_prod * proto_prod
                  sqrt_one_minus_alpha_prod=sqrt_one_minus_alpha_prod * proto_prod
                  init_latents_proper = sqrt_alpha_prod * init_latents + sqrt_one_minus_alpha_prod * noise
                  latents = init_latents_proper * area_mask    +    latents * (1-area_mask)
                  print('post process 1')
                else:
                  proto_prod = torch.ones(1,1,1,1, device=torch_device, dtype=torch.float16)
                  sqrt_alpha_prod = scheduler.alphas_cumprod[t-1] ** 0.5
                  sqrt_one_minus_alpha_prod = (1 - scheduler.alphas_cumprod[t-1]) ** 0.5
                  sqrt_alpha_prod=sqrt_alpha_prod * proto_prod
                  sqrt_one_minus_alpha_prod=sqrt_one_minus_alpha_prod * proto_prod
                  init_latents_proper = sqrt_alpha_prod * init_latents + sqrt_one_minus_alpha_prod * noise
                  # init_latents_proper = scheduler.add_noise(init_latents, noise, t-1)
                  latents = (1-area_mask)* (1-layer_mask) * latents + (1-(1-area_mask)* (1-layer_mask)) * init_latents_proper # In / Out
                  print('post process 2')
              else:
                print('this is last')
                # if i >= int((1-overcoat_ratio)*num_inference_steps):
                  # init_latents_proper = scheduler.add_noise(init_latents, noise, t-1)
                proto_prod = torch.ones(1,1,1,1, device=torch_device, dtype=torch.float16)
                sqrt_alpha_prod = scheduler.alphas_cumprod[t-1] ** 0.5
                sqrt_one_minus_alpha_prod = (1 - scheduler.alphas_cumprod[t-1]) ** 0.5
                sqrt_alpha_prod=sqrt_alpha_prod * proto_prod
                sqrt_one_minus_alpha_prod=sqrt_one_minus_alpha_prod * proto_prod
                init_latents_proper = sqrt_alpha_prod * init_latents + sqrt_one_minus_alpha_prod * noise
                last_latents_proper = scheduler.add_noise(last_latents, noise, t-1)
                # latents = init_latents_proper * (last_mask+area_mask-1)    +    latents * (1-area_mask) + last_latents_proper * (1-last_mask)
                latents = init_latents_proper * (area_mask-layer_mask*area_mask)    +    latents * (1-area_mask) + last_latents_proper * (layer_mask* area_mask)
                # lms.append(init_latents_proper * (last_mask+area_mask-1))
                # lms.append(latents * (1-area_mask) )
                # lms.append(last_latents_proper * (1-last_mask))
                print('post process 1')
                # else:
                #   proto_prod = torch.ones(1,1,1,1, device=torch_device)
                #   sqrt_alpha_prod = scheduler.alphas_cumprod[t-1] ** 0.5
                #   sqrt_one_minus_alpha_prod = (1 - scheduler.alphas_cumprod[t-1]) ** 0.5
                #   sqrt_alpha_prod=sqrt_alpha_prod * proto_prod
                #   sqrt_one_minus_alpha_prod=sqrt_one_minus_alpha_prod * proto_prod
                #   init_latents_proper = sqrt_alpha_prod * init_latents + sqrt_one_minus_alpha_prod * noise
                #   last_latents_proper = scheduler.add_noise(last_latents, noise, t-1)
                #   # init_latents_proper = scheduler.add_noise(init_latents, noise, t-1)
                #   # latents = (1-area_mask)* (1-layer_mask) * latents + (last_mask-(1-area_mask)* (1-layer_mask)) * init_latents_proper + last_latents_proper * (1-last_mask) # In / Out
                #   latents = init_latents_proper * (layer_mask-(1-last_mask))    +    latents * (1-last_mask) *(1-area_mask) + last_latents_proper * (1-last_mask)
                #   print('post process 2')
                
              # when below overcoat ratio
            else:
              if last_mask==None:
                latents = init_latents * area_mask    +    latents * (1-area_mask)
              else:
                # latents = init_latents* (last_mask+area_mask-1)    +    latents * (1-area_mask) + last_latents * (1-last_mask)
                latents = init_latents * (area_mask-layer_mask*area_mask)    +    latents * (1-area_mask) + last_latents * (layer_mask*area_mask)
              print('post process 3')
            print('post process', time.time()-now)

            latents_list[message['stroke_id']].append(latents)
            dir_prompt_list[message['stroke_id']].append(directional_prompt_inputs_proto[message['stroke_id']])
            prompt_list[message['stroke_id']].append(prompt_inputs_proto[message['stroke_id']])
            guidance_scale_list[message['stroke_id']].append(guidance_scale[message['stroke_id']])
            prompts_whole_list[message['stroke_id']].append(prompts_whole_proto[message['stroke_id']])

            int_thread = Thread(target=sendoutIntermediate, args = (message['gen_tick']+i, layer_img_o, area_img_o, message['area_img'], message['stroke_id'], gen_duration))
            int_thread.start()
      except:
        emit('gen_failed', {'data':'gen failed!'})



    @copy_current_request_context
    def sendoutIntermediate(gen_tick, layer_img_o, area_img_o, area_img_d, stroke_id, gen_duration):
      idx = gen_tick
      ta = time.time()
      with torch.no_grad():
          
          
        if gen_stop[stroke_id]:
          return
        print('before emit', time.time()-ta)
        latents = latents_list[stroke_id][idx]
        dir_prompts_ = dir_prompt_list[stroke_id][idx]
        prompts_ = prompt_list[stroke_id][idx]
        prompts_whole_ = prompts_whole_list[stroke_id][idx]
        guidance_scale_ = guidance_scale_list[stroke_id][idx]
        print(latents.shape)
        
        output_img = 1 / 0.18215 * latents
        output_img = vae.decode(output_img).sample
        output_img = (output_img / 2 + 0.5).clamp(0, 1)
        output_img = output_img.cpu().permute(0, 2, 3, 1).numpy()
        output_img = numpy_to_pil(output_img)[0]
        output_img = output_img.resize((layer_img_o.size[0], layer_img_o.size[1]), resample=Image.LANCZOS)
        output_array = np.asarray(output_img)
        output_array = np.copy(output_array)
        latents_rt = latents.tolist()
        # print('within3', time.time()-now)

        area_array = np.asarray(area_img_o)
        area_array = np.where(area_array==255, 255, 0)
        # print(area_array.shape, gaussian.shape)
        output_array[:,:,3] = area_array[:,:,3]
        # print('within4', time.time()-now)
        output_img = Image.fromarray(output_array)
        # print('within5', time.time()-now)
        
        # print('here?')
        buffered = BytesIO()
        output_img.save(buffered, format="PNG")
        output_img_send = base64.b64encode(buffered.getvalue()).decode("utf-8")
        print('lets emit', time.time()-ta)
        # emit('test', {'data':'test'+str(idx)})
        idx = idx+1
        emit('intermediate_gen', {'data':area_img_d.split(",",1)[0]+','+output_img_send, 'gen_tick':idx, 'stroke_id': stroke_id, 'latents': latents_rt, 'selected_prompts': prompts_, 'prompts':prompts_whole_, 'directional_prompts': dir_prompts_, 'guidance_scale': guidance_scale_})
        print('thread end!')

    threads[message['stroke_id']] = Thread(target=handle_message_thread , args = [message])
    threads[message['stroke_id']].start()
    emit('test', {'data':'gen started!'})


    





# if __name__ == '__main__':
# socketio.run(app, host='0.0.0.0', debug=False, port=5001)
http_server = WSGIServer(('',5001), app, handler_class=WebSocketHandler)
http_server.serve_forever()



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/gevent/threadpool.py", line 163, in _before_run_task
    _sys.settrace(_get_thread_trace())


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/gevent/threadpool.py", line 168, in _after_run_task
    _sys.settrace(None)



someone connected to websocket: noIYz0mrO2VKjgrDAAAB
368 512
(512, 704)
1.7125086784362793
1.7143683433532715
1.7522854804992676
tensor([950, 900, 850, 800, 750, 700, 650, 600, 550, 500, 450, 400, 350, 300,
        250, 200, 150, 100,  50,   0]) tensor([950, 900, 850, 800, 750, 700, 650, 600, 550, 500, 450, 400, 350, 300,
        250, 200, 150, 100,  50,   0])
1.7544329166412354
0 start 7
unet pipeline0 1.7553260326385498 True <class 'torch.Tensor'> torch.Size([]) True
torch.cuda.HalfTensor
unet pipeline1 1.907485008239746
unet pipeline2 1.9094815254211426
post process 2
post process 1.9109878540039062
before emit 1.9550323486328125e-05
torch.Size([1, 4, 88, 64])
1 start 7
unet pipeline0 1.9134571552276611 True <class 'torch.Tensor'> torch.Size([]) True
torch.cuda.HalfTensor
unet pipeline1 2.5807595252990723
unet pipeline2 2.5835320949554443
post process 2
post process 2.5851552486419678
before emit 1.9073486328125e-05
torch.Size([1, 4, 88, 64])
2 start 7
unet pipeline0 2.5868983268737

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 172, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 474, in __exit__
    self.auto_pop(exc_value)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 460, in auto_pop
    self.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 447, in pop
    app_ctx.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 253, in pop
    assert rv is self, f"Popped wrong app context.  ({rv!r} instead of {self!r})"
AssertionError: Popped wrong app context.  (<flask.ctx.AppContext object at 0x7f592e089bd0> instead of <flask.ctx.AppContext object at 0x7f592e0f4450>)



torch.cuda.HalfTensor
unet pipeline1 3.2181994915008545
unet pipeline2 3.220879554748535
post process 2
post process 3.222484588623047
before emit 1.811981201171875e-05
torch.Size([1, 4, 88, 64])
3 start 7
unet pipeline0 3.2283055782318115 True <class 'torch.Tensor'> torch.Size([]) True
lets emit 0.7296297550201416
thread end!


Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 172, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 474, in __exit__
    self.auto_pop(exc_value)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 460, in auto_pop
    self.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 447, in pop
    app_ctx.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 253, in pop
    assert rv is self, f"Popped wrong app context.  ({rv!r} instead of {self!r})"
AssertionError: Popped wrong app context.  (<flask.ctx.AppContext object at 0x7f592e0f4450> instead of <flask.ctx.AppContext object at 0x7f592e0922d0>)



...

torch.cuda.HalfTensor
unet pipeline1 8.672089338302612
unet pipeline2 8.674787521362305
post process 3
post process 8.676310300827026
before emit 1.9073486328125e-05
torch.Size([1, 4, 64, 64])
lets emit 0.6221089363098145
thread end!


Exception in thread Thread-52:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 172, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 474, in __exit__
    self.auto_pop(exc_value)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 460, in auto_pop
    self.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 447, in pop
    app_ctx.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 253, in pop
    assert rv is self, f"Popped wrong app context.  ({rv!r} instead of {self!r})"
AssertionError: Popped wrong app context.  (<flask.ctx.AppContext object at 0x7f586a24dad0> instead of <flask.ctx.AppContext object at 0x7f586a3bdd50>)



lets emit 0.5084507465362549
thread end!


Exception in thread Thread-53:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 172, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 474, in __exit__
    self.auto_pop(exc_value)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 460, in auto_pop
    self.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 447, in pop
    app_ctx.pop(exc)
  File "/usr/local/lib/python3.7/dist-packages/flask/ctx.py", line 253, in pop
    assert rv is self, f"Popped wrong app context.  ({rv!r} instead of {self!r})"
AssertionError: Popped wrong app context.  (<flask.ctx.AppContext object at 0x7f586a3bdd50> instead of <flask.ctx.AppContext object at 0x7f592e02a810>)



KeyboardInterrupt: ignored